# Artificial Intelligence Nanodegree
## Machine Translation Project

In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. The completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import tensorflow
tensorflow.__version__

'1.3.0'

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12913954108617890371
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357564416
locality {
  bus_id: 1
}
incarnation: 15392779437507797026
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


In [5]:
import keras
keras.__version__

'2.0.9'

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [6]:
# Load English data
english_sentences = helper.load_data('translation_data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('translation_data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [7]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
Convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Each sentence is converted into a sequence of words ids using Keras's [`Tokenizer`] function to tokenize `english_sentences` and `french_sentences` in the cell below.

In [9]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    print(x)
    tkn = Tokenizer(lower=True)
    tkn.fit_on_texts(x)
    return tkn.texts_to_sequences(x),tkn

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

['The quick brown fox jumps over the lazy dog .', 'By Jove , my quick study of lexicography won a prize .', 'This is a short sentence .']
['The quick brown fox jumps over the lazy dog .', 'By Jove , my quick study of lexicography won a prize .', 'This is a short sentence .']
{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

All the English sequences can have the same length and all the French sequences can have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`] function.

In [10]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    maxlen = max(len(i) for i in x) if length is None else length
    padded_x = pad_sequences(x,maxlen=maxlen,padding='post')
    return padded_x
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [11]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
We will experiment with various neural network architectures starting with relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation. 
(This function will also help to better understand the output of the neural network)

In [12]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {_id: word for word, _id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data. In this model, a RNN will translates English to French.

In [14]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.009
    inputs = Input(input_shape[1:])
    rnn_layer = GRU(512,return_sequences=True)(inputs)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn_layer)
    activation = Activation('softmax')(logits)
    model = Model(inputs,activation)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

test_sentence = [i[0] for i in tmp_x[0]]
# print(test_sentence)
index_to_words = {id: word for word, id in english_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'
english_sentence = [index_to_words[i] for i in test_sentence]
print('English:',english_sentence)

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=40, validation_split=0.2)

# Print prediction(s)
print('French:',logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
# best train acc = 84.8%, val accuracy = 86.86%

English: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', 'and', 'it', 'is', 'snowy', 'in', 'april', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Train on 110288 samples, validate on 27573 samples
Epoch 1/40
110288/110288 [==============================] - 19s 170us/step - loss: 1.5845 - acc: 0.5937 - val_loss: nan - val_acc: 0.6615
Epoch 2/40
110288/110288 [==============================] - 18s 166us/step - loss: 1.0539 - acc: 0.6679 - val_loss: nan - val_acc: 0.6813
Epoch 3/40
110288/110288 [==============================] - 18s 166us/step - loss: 0.9358 - acc: 0.6894 - val_loss: nan - val_acc: 0.6999
Epoch 4/40
110288/110288 [==============================] - 18s 166us/step - loss: 0.8776 - acc: 0.7019 - val_loss: nan - val_acc: 0.7239
Epoch 5/40
110288/110288 [==============================] - 18s 166us/step - loss: 0.8370 - acc: 0.7082 - val_loss: nan - val_acc: 0.7173
Epoch 6/40
110288/110288 [==============================] - 18s 166us

### Model 2: Embedding
![RNN](images/embedding.png)
There's a better representation of words than simply converting them to IDs, called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

This model is a RNN model using embedding.

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    print('Embed_input_shape',input_shape[1:])
    print('Output sequence length',output_sequence_length)
    print('English vocab size:',english_vocab_size,'French vocab size:',french_vocab_size)
    
    learning_rate = 0.008
    hidden_layer = 256
    
    embed_input = Input(input_shape[1:])
    embedded = Embedding(english_vocab_size,hidden_layer, input_length=output_sequence_length)(embed_input)
    rnn_layer = GRU(hidden_layer,return_sequences=True)(embedded)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn_layer)
    activation = Activation('softmax')(logits)
    model = Model(embed_input,activation)
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])
    print(model.summary())
    return model
tests.test_embed_model(embed_model)

embedded_rnn_model = embed_model(tmp_x.shape,max_french_sequence_length,english_vocab_size, french_vocab_size)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
test_sentence = [i[0] for i in tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))[0]]
index_to_words = {id: word for word, id in english_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'
english_sentence = [index_to_words[i] for i in test_sentence]
print('English:',english_sentence)

# Train the neural network
embedded_rnn_model.fit(tmp_x,preproc_french_sentences,batch_size=1024,epochs=30,validation_split=0.2)
result = logits_to_text(embedded_rnn_model.predict(tmp_x[:1])[0],french_tokenizer)
# print prediction(s)
print('French:',logits_to_text(embedded_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
# best train accuracy = 93.65%, val accuracy = 93.42%

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.008
    hidden_layer = 384
#     print('input_shape',input_shape)
    input_layer = Input(shape=input_shape[1:],
                        dtype='float32',
                        name='input_layer')
    x = Bidirectional(GRU(hidden_layer, return_sequences=True))(input_layer)
    output_layer = TimeDistributed(Dense(french_vocab_size,
                         activation='softmax',
                         name='output_layer'))(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])
    print(model.summary())
    return model
tests.test_bd_model(bd_model)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

test_sentence = [i[0] for i in tmp_x[0]]
# print(test_sentence)
index_to_words = {id: word for word, id in english_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'
english_sentence = [index_to_words[i] for i in test_sentence]
print('English:',english_sentence)

# Train the neural network
bi_dir_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bi_dir_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=70, validation_split=0.2)

# Print prediction(s)
print('French:',logits_to_text(bi_dir_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
# best train accuracy = 87.14%, val accuracy = 87.36%

input_shape (137861, 21, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21, 1)             0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 21, 768)           889344    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 344)           264536    
Total params: 1,153,880
Trainable params: 1,153,880
Non-trainable params: 0
_________________________________________________________________
None
English: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', 'and', 'it', 'is', 'snowy', 'in', 'april', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
input_shape (137861, 21, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (

### Model 4: Encoder-Decoder
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [22]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.005
    hidden_layer = 256
    # training encoder
    encoder_input = Input(shape=input_shape[1:],dtype='float32',name='input_layer')
    encoder_outputs = GRU(hidden_layer,name='encoder')(encoder_input)
    rp_enc_output = RepeatVector(output_sequence_length)(encoder_outputs)
    # training decoder 
    decoder = GRU(hidden_layer,return_sequences=True,name='decoder')
    decoder_output = decoder(rp_enc_output)
    decoder_output = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(decoder_output)
    
    model = Model(encoder_input,decoder_output,name='training_model')
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])    
    
    return model
tests.test_encdec_model(encdec_model)

# Reshaping the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
enc_dec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(enc_dec_model.summary())
enc_dec_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
# best train accuracy = 77.73%, val accuracy = 79.27

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21, 1)             0         
_________________________________________________________________
encoder_2 (GRU)              (None, 256)               198144    
_________________________________________________________________
repeat_vector_17 (RepeatVect (None, 21, 256)           0         
_________________________________________________________________
decoder (GRU)                (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_15 (TimeDis (None, 21, 344)           88408     
Total params: 680,536
Trainable params: 680,536
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 20s 185us/step - los

### Model 5 
Combining all knowledge gained from the previous experiments, it seems the best model should include an embedding layer, should be bidirectional and have an encoder-decoder architecture.

In [40]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.005
    hidden_layer = 256
    print(input_shape)
    inputs = Input(shape=input_shape[1:])
    embedded_inputs = Embedding(input_dim=english_vocab_size,output_dim=hidden_layer,
                                input_length=output_sequence_length)(inputs)
#     encoder_output = Bidirectional(GRU(hidden_layer,return_sequences=True,name='encoder_1'))(embedded_inputs)
    encoder_output = Bidirectional(GRU(hidden_layer,name='encoder_2'))(embedded_inputs)
    rp_enc_output = RepeatVector(output_sequence_length)(encoder_output)
    
    decoder = Bidirectional(GRU(hidden_layer,return_sequences=True,name='encoder'))(rp_enc_output)
    final_output = TimeDistributed(Dense(french_vocab_size,activation='softmax',name='output_layer'))(decoder)
    
    model = Model(inputs=inputs,outputs=final_output)
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])
    
    return model
tests.test_model_final(model_final)

print('Final Model Loaded')

(137861, 15)
Final Model Loaded


## Prediction

In [53]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
    model = model_final(tmp_x.shape, y.shape[1], len(x_tk.word_index), len(y_tk.word_index))
    
    print(model.summary())
    model.fit(tmp_x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    ## 
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    tmp_sentences = pad(sentences, max_french_sequence_length)
    predictions = model.predict(tmp_sentences, len(tmp_sentences))
    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(137861, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_29 (Embedding)     (None, 21, 256)           50944     
_________________________________________________________________
bidirectional_62 (Bidirectio (None, 512)               787968    
_________________________________________________________________
repeat_vector_44 (RepeatVect (None, 21, 512)           0         
_________________________________________________________________
bidirectional_63 (Bidirectio (None, 21, 512)           1181184   
_________________________________________________________________
time_distributed_41 (TimeDis (None, 21, 344)           176472    
Total params: 2,196,568
Trainable params: 2,196,568
Non-trainable params: 0
_____________________________________________________

## Future Enhancements

This project focuses on learning various network architectures for machine translation, but we did not evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Using [`sklearn.model_selection.train_test_split()`] function to create separate training & test datasets we can then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set.